In [1]:
using ForwardDiff
#using PyPlot
using JuliaFEM

using JuliaFEM.Core: Seg2, Quad4, Hex8, get_connectivity,
                     assemble, Elasticity

using JuliaFEM.Core: get_integration_points

In [7]:
nodes = Dict{Int64, Vector{Float64}}(
    1 => [0.0, 0.0],
    2 => [1.0, 0.0],
    3 => [1.0, 1.0],
    4 => [0.0, 1.0])

function set_geometry!(element, nodes)
    element["geometry"] = Vector{Float64}[nodes[i] for i in get_connectivity(element)]
end
element1 = Quad4([1, 2, 3, 4])
set_geometry!(element1, nodes)
element1["youngs modulus"] = 9000.0
element1["poissons ratio"] = 0.25
element1["displacement"] = [[1.0 1.0]
    [0.0 0.0]
    [0.0 0.0]
    [0.0 0.0]]

free_dofs = Int64[3, 5, 6, 8]

time = 0.0

element1("displacement", time)

JuliaFEM.Core.Field{JuliaFEM.Core.Discrete,JuliaFEM.Core.Constant,JuliaFEM.Core.TimeInvariant}(4x2 Array{Float64,2}:
 1.0  1.0
 0.0  0.0
 0.0  0.0
 0.0  0.0)

In [8]:
time = 0.0

u_max = 0.1
dim = 2
nnodes = 4

for element in [element1]
    
    u = element1("displacement", time)
    u = reshape(u, dim, nnodes)
    u = Field([u[:,i] for i=1:nnodes])
    
    for ip in get_integration_points(element)

        grad = element(ip, time, Val{:grad})
        gradu = grad*u

        # kinematics
        #F = I + gradu
        #E = 1/2*(F'*F - I)
        E = 1/2*(gradu + gradu') 
        ee = zeros(4,4)
        for idx=1:4
            ee[:, 1] = E[idx]
        end
        println(ee)
    end
  
end

LoadError: LoadError: MethodError: `reshape` has no method matching reshape(::JuliaFEM.Core.Field{JuliaFEM.Core.Discrete,JuliaFEM.Core.Constant,JuliaFEM.Core.TimeInvariant}, ::Int64, ::Int64)
Closest candidates are:
  reshape(!Matched::AbstractArray{T,N}, ::Int64...)
while loading In[8], in expression starting on line 7

In [30]:
ii = zeros(4,4)
ii[1, :]

1x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0

In [3]:
function assemble(problem::Problem{Elasticity}, element::Element, time::Real, ::Type{Val{:forwarddiff}})

    dim = get_unknown_field_dimension(problem)
    nnodes = size(element, 2)

    function get_residual_vector(u::Vector)
        u = reshape(u, dim, nnodes)
        u = Field([u[:,i] for i=1:nnodes])
        r = zeros(dim, nnodes)

        for ip in get_integration_points(element)

            JT = transpose(get_jacobian(element, ip, time))
            n, m = size(JT)
            if n == m
                w = ip.weight*det(JT)
            elseif m == 1
                w = ip.weight*norm(JT)
            elseif m == 2
                w = ip.weight*norm(cross(JT[:,1], JT[:,2]))
            else
                error("jacobian $JT")
            end

            # calculate internal forces
            if haskey(element, "youngs modulus") && haskey(element, "poissons ratio")
                grad = element(ip, time, Val{:grad})
                gradu = grad*u

                # kinematics
                F = I + gradu
                E = 1/2*(F'*F - I)

                # material
                young = element("youngs modulus", ip, time)
                poisson = element("poissons ratio", ip, time)
                mu = young/(2*(1+poisson))
                lambda = young*poisson/((1+poisson)*(1-2*poisson))
                if problem.properties.formulation == :plane_stress
                    lambda = 2*lambda*mu/(lambda + 2*mu)  # <- correction for plane stress
                end

                # stress
                S = lambda*trace(E)*I + 2*mu*E

                r += w*F*S*grad
            end

            # calculate external forces - volume load
            if haskey(element, "displacement load")
                basis = element(ip, time)
                b = element("displacement load", ip, time)
                r -= w*b*basis
            end

            # external forces - surface traction force
            if haskey(element, "displacement traction force")
                basis = element(ip, time)
                T = element("displacement traction force", ip, time)
                r -= w*T*basis
            end

        end

        return vec(r)

    end

    field = element("displacement", time)
    Kt, allresults = ForwardDiff.jacobian(get_residual_vector, vec(field),
                     AllResults, cache=autodiffcache)
    f = -ForwardDiff.value(allresults)
    return Kt, f
end

JuliaFEM.Core.Elasticity(:continuum,true,false)

In [46]:
using JuliaFEM

body = Problem(Elasticity, "block", 2)
body.properties.formulation = :plane_stress
nodes = Dict{Int64, Vector{Float64}}(
    1 => [0.0, 0.0],
    2 => [1.0, 0.0],
    3 => [1.0, 1.0],
    4 => [0.0, 1.0])

element = Element(Quad4, [1,2,3,4])
update!(element, "geometry", nodes)
element["youngs modulus"] = 9000.0
element["poissons ratio"] = 1/3

# dirichlet boundary conditions
boundary2 = Element(Seg2, [2,3])
update!(boundary2, "geometry", nodes)
boundary2["displacement traction force 2"] = [10.0, 0.0]

push!(body, element)
push!(body, boundary2)


# dirichlet boundary conditions
boundary_problem_1 = Problem(Dirichlet, "bc", 2, "displacement")
boundary = Element(Seg2, [1, 4])
update!(boundary, "geometry", nodes)
boundary["displacement 1"] = 0.0
boundary["displacement 2"] = 0.0
push!(boundary_problem_1, boundary)


solver = Solver("solve block problem")

push!(solver, body, boundary_problem_1)
call(solver)

disp = element("displacement", [1.0, 1.0], 0.0)
info("displacement at tip: $disp")

INFO: Starting nonlinear iteration #1
INFO: solving linear system of 2 problems.
INFO: UMFPACK: solved in 0.0 seconds. norm = 0.003993254146735212
INFO: block: incremental formulation, adding increment to solution vector
INFO: bc: total formulation, replacing solution vector with new values
INFO: Details for problem block
INFO: Norm: 0.003993254146735212
INFO: Norm change: 0.003993254146735212
INFO: Has converged? false
INFO: Starting nonlinear iteration #2
INFO: solving linear system of 2 problems.
INFO: UMFPACK: solved in 0.0 seconds. norm = 5.263468075252658e-6
INFO: block: incremental formulation, adding increment to solution vector
INFO: bc: total formulation, replacing solution vector with new values
INFO: Details for problem block
INFO: Norm: 0.0039942305163527415
INFO: Norm change: 5.263468075252619e-6
INFO: Has converged? true
INFO: Converged in 2 iterations.
INFO: displacement at tip: [-0.0010737519668183434
 0.0024687795720146296]


In [44]:
dump(full(boundary_problem_2.assembly.C2))

Array(Float64,(0,0)) 0x0 Array{Float64,2}
